# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import glob 
import os 
import math
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter, MonthLocator

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("Output/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,remire-montjoly,4.9167,-52.2667,28.02,74,75,3.09,GF,1708641268
1,1,i-n-salah,27.1935,2.4607,14.52,37,0,4.43,DZ,1708641269
2,2,fort mcmurray,56.7268,-111.3810,3.77,52,0,1.54,CA,1708641269
3,3,puerto natales,-51.7236,-72.4875,6.48,78,45,6.74,CL,1708640978
4,4,puerto ayora,-0.7393,-90.3518,30.98,93,93,1.34,EC,1708640979
5,5,anadyr,64.7500,177.4833,-13.72,72,40,9.00,RU,1708640979
6,6,al matariyah,31.1829,32.0311,16.97,70,100,4.95,EG,1708640979
7,7,plettenberg bay,-34.0527,23.3716,16.78,75,75,1.47,ZA,1708640979
8,8,hithadhoo,-0.6000,73.0833,28.13,77,100,1.56,MV,1708640980
9,9,invercargill,-46.4000,168.3500,17.29,49,40,3.22,NZ,1708640955


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

pip install geoviews

In [24]:
l = len(city_data_df["City"])

In [25]:
# Configure the map plot
from matplotlib.colors import LinearSegmentedColormap
colors  = ['green', 'darkorange', 'firebrick'] 
cm      = LinearSegmentedColormap.from_list("Custom", colors, N=100)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
plt.figure(figsize=(100,100))
m3 = city_data_df.hvplot.points(x='Lng', y='Lat', geo = True, tiles = "OSM", marker='circle', 
                                size=city_data_df["Humidity"], frame_height=1000, fontsize=18, 
                                xlabel="Longitude", ylabel="Latitude", alpha=10000000.0, cmap=cm, color=np.random.randint(0,l,l),
                                fill_alpha=15, label="City")
m3

:Overlay
   .Tiles.I     :Tiles   [x,y]
   .Points.City :Points   [Lng,Lat]   (_color,_size)

<Figure size 10000x10000 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 35) & (city_data_df['Wind Speed'] < 2.5)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,puerto ayora,-0.7393,-90.3518,30.98,93,93,1.34,EC,1708640979
7,7,plettenberg bay,-34.0527,23.3716,16.78,75,75,1.47,ZA,1708640979
8,8,hithadhoo,-0.6000,73.0833,28.13,77,100,1.56,MV,1708640980
15,15,kerikeri,-35.2268,173.9474,23.28,79,55,1.46,NZ,1708640899
22,22,lorengau,-2.0226,147.2712,28.12,79,100,0.90,PG,1708640983
...,...,...,...,...,...,...,...,...,...,...
555,555,neelankarai,12.9455,80.2575,27.26,89,40,1.54,IN,1708641237
570,570,kingaroy,-26.5333,151.8333,27.29,62,30,1.56,AU,1708641245
575,575,les avirons,-21.2418,55.3394,25.03,94,75,2.06,RE,1708641247
577,577,ilheus,-14.7889,-39.0494,26.90,89,40,0.00,BR,1708641248


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  city_data_df[['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
4,4,puerto ayora,EC,-0.7393,-90.3518,93,
7,7,plettenberg bay,ZA,-34.0527,23.3716,75,
8,8,hithadhoo,MV,-0.6000,73.0833,77,
15,15,kerikeri,NZ,-35.2268,173.9474,79,
22,22,lorengau,PG,-2.0226,147.2712,79,
...,...,...,...,...,...,...,...
555,555,neelankarai,IN,12.9455,80.2575,89,
570,570,kingaroy,AU,-26.5333,151.8333,62,
575,575,les avirons,RE,-21.2418,55.3394,94,
577,577,ilheus,BR,-14.7889,-39.0494,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
#from config import geoapify_key
#geoapify_key = "eb43efe87ad84d32a9e4cb0f86666dfc"

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"]  = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
plettenberg bay - nearest hotel: Milkwood Manor
hithadhoo - nearest hotel: Pebbles Inn
kerikeri - nearest hotel: Avalon Resort
lorengau - nearest hotel: Seeadler Bay Hotel
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
poxoreo - nearest hotel: No hotel found
juazeirinho - nearest hotel: No hotel found
mbandaka - nearest hotel: Hôtel Epervier
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
waitangi - nearest hotel: Hotel Chathams
extrema - nearest hotel: Hotel San Marco
papatowai - nearest hotel: No hotel found
cao bang - nearest hotel: SONG HIEN HOTEL
bhamo - nearest hotel: Winlight Hotel
port saint john's - nearest hotel: Outback Inn
benguela - nearest hotel: Hotel Moibela
poso - nearest hotel: Nusantara
borba - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
chimoio - nearest hotel: Hotel Amirana
nch

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
4,4,puerto ayora,EC,-0.7393,-90.3518,93,Hostal La Mirada De Solitario George
7,7,plettenberg bay,ZA,-34.0527,23.3716,75,Milkwood Manor
8,8,hithadhoo,MV,-0.6000,73.0833,77,Pebbles Inn
15,15,kerikeri,NZ,-35.2268,173.9474,79,Avalon Resort
22,22,lorengau,PG,-2.0226,147.2712,79,Seeadler Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
from matplotlib.colors import LinearSegmentedColormap
colors  = ['green', 'darkorange', 'firebrick'] 
cm      = LinearSegmentedColormap.from_list("Custom", colors, N=100)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
plt.figure(figsize=(100,100))
m4 = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, tiles = 'OSM', frame_width = 700, 
                            frame_height = 500, color='City', alpha=0.6, hover=True,
                            hover_cols='all', use_index=False, s='Humidity',
                            scale= 1.2)
m4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Country,Hotel Name)

<Figure size 10000x10000 with 0 Axes>